# Flight Delay Analysis

In [62]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from typing import Optional
%matplotlib inline

In [ ]:
# load data
path = 'notebook_data/*.csv'

# Use glob to get all CSV file paths
all_files = glob.glob(path)

# Read and concatenate all CSV files into a single DataFrame
df = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

In [ ]:
df.head(10)

In [ ]:
df[df["ORIGIN_CITY_NAME"] == "Milwaukee, WI"]

In [ ]:
df[(df["ORIGIN_CITY_NAME"] == "Milwaukee, WI") & (df["DAY_OF_MONTH"] == 24)]

In [ ]:
# plot the number of flights by day
def get_num_flights(day, city):
    return len(df[(df["ORIGIN_CITY_NAME"] == city) & (df["DAY_OF_MONTH"] == day)])

In [ ]:
cities = ['Milwaukee, WI', 'San Diego, CA', 'San Francisco, CA']

for city in cities:
    days = range(1, 32)
    num_flights = [get_num_flights(day, city) for day in days]
    
    plt.figure(figsize=(12, 6))
    plt.plot(days, num_flights, 'b-', label='Number of Flights')

    # Mark day 24 with a red dot
    plt.plot(24, get_num_flights(24, city), 'ro', markersize=10, label='XMas Eve')

    # Customize the plot
    plt.title(f'Number of Flights by Day of Month {city} December 2018-2023', fontsize=16)
    plt.xlabel('Day of Month', fontsize=12)
    plt.ylabel('Number of Flights', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend()

    # Set x-axis ticks to show all days
    plt.xticks(days)

    # Show the plot
    plt.tight_layout()
    plt.show()

In [ ]:
def compute_delay_probability_with_cancellation(filtered_df, delay, city):
    filtered_df = filtered_df[filtered_df["ORIGIN_CITY_NAME"] == city]
    num_delayed_or_cancelled = len(filtered_df[(filtered_df["ARR_DELAY"] >= delay) | (filtered_df["CANCELLED"] == True) | (filtered_df["DIVERTED"] == True)])
    return num_delayed_or_cancelled / len(filtered_df)

In [ ]:
def compute_delay_probability_without_cancellation(filtered_df, delay, city):
    filtered_df = filtered_df[filtered_df["ORIGIN_CITY_NAME"] == city]
    not_cancelled = filtered_df[(filtered_df["CANCELLED"] == False) & (filtered_df["DIVERTED"] == False)]
    num_delayed = len(not_cancelled[(not_cancelled["ARR_DELAY"] >= delay)])
    return num_delayed / len(not_cancelled)

In [ ]:
cities = ['Milwaukee, WI', 'San Diego, CA', 'San Francisco, CA']

for city in cities:
    filtered_df = df[df["ORIGIN_CITY_NAME"] == city]
    xmas_eve_df = filtered_df[filtered_df["DAY_OF_MONTH"] == 24]
    max_delay = 250  # override

    delays = np.arange(0, max_delay + 1)

    december_probabilities = [compute_delay_probability_without_cancellation(filtered_df, delay, city) for delay in delays]
    xmas_eve_probabilities = [compute_delay_probability_without_cancellation(xmas_eve_df, delay, city) for delay in delays]

    plt.figure(figsize=(14, 7))

    plt.plot(delays, december_probabilities, 'b-', label='December')
    plt.plot(delays, xmas_eve_probabilities, 'r-', label='Xmas Eve')

    december_prob_1h = december_probabilities[60]
    xmas_eve_prob_1h = xmas_eve_probabilities[60]

    plt.axhline(y=december_prob_1h, color='b', linestyle=':', xmin=0)
    plt.axhline(y=xmas_eve_prob_1h, color='r', linestyle=':', xmin=0)

    plt.plot(60, december_prob_1h, 'bo', markersize=8)
    plt.plot(60, xmas_eve_prob_1h, 'ro', markersize=8)

    plt.annotate(f'{december_prob_1h:.2%}', (60, december_prob_1h), xytext=(0, 10), 
                 textcoords='offset points', ha='center', va='bottom', color='b')
    plt.annotate(f'{xmas_eve_prob_1h:.2%}', (60, xmas_eve_prob_1h), xytext=(0, 10), 
                 textcoords='offset points', ha='center', va='bottom', color='r')

    plt.title(f'Delay Probability (without Cancellations) vs. Delay Duration {city} 2018-2023', fontsize=16)
    plt.xlabel('Delay (hours:minutes)', fontsize=12)
    plt.ylabel('Probability', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)

    plt.ylim(0, 0.5)
    plt.yticks(np.arange(0, 0.51, 0.1))
    plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.0%}'.format(y)))

    plt.legend(fontsize=10)

    def format_time(x, pos):
        hours = int(x // 60)
        minutes = int(x % 60)
        return f'{hours:02d}:{minutes:02d}'

    plt.gca().xaxis.set_major_formatter(plt.FuncFormatter(format_time))
    plt.xticks(np.arange(0, max_delay + 1, 30), rotation=45)

    plt.xlim(0, max_delay * 1.05)

    plt.tight_layout()
    plt.show()

## Analysis on Full Data
We load the full data and investigate distributions of delays conditioned on various variables. Our goal is to find a generalized parameterized model for the delay distribution so that we can store the parameters for this model for each conditional.

First, we look to prune the raw data (columns and rows) to what we minimally need. This logic will be ported to a file to cache pruned data. Then, we carry out analysis on the resulting df.

In [ ]:
data_dir = "data/"

In [ ]:
# read data frame from all csv files in the data_dir
def read_data(data_dir):
    path = os.path.join(data_dir, '*.csv')
    all_files = glob.glob(path)
    return pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

In [ ]:
df = read_data(data_dir)

In [ ]:
df.head(10)

In [ ]:
# add a new column bucketing the departure time into morning, afternoon, evening, and night
# range left inclusive, right exclusive
BUCKETS = {
    'morning': (600, 1200),
    'afternoon': (1200, 1800),
    'evening': (1800, 2400),
    'night': (0, 600),
}

def get_bucket(hour: Optional[float]) -> Optional[str]:
    if hour is None:
        return None
    for bucket, (start, end) in BUCKETS.items():
        if start <= hour < end:
            return bucket

df['DEP_TIME_BUCKET'] = df['CRS_DEP_TIME'].apply(get_bucket)

In [ ]:
df.head(10)

In [ ]:
# filter out any cancelled or diverted flights
filtered_df = df[(df['CANCELLED'] == 0) & (df['DIVERTED'] == 0)]

In [ ]:
len(filtered_df)

In [ ]:
# only keep the columns we need
columns_needed = [
    'OP_CARRIER_AIRLINE_ID',
    'ORIGIN_AIRPORT_ID',
    'DEST_AIRPORT_ID',
    'DEP_TIME_BUCKET',
    'ARR_DELAY',
]

filtered_df = filtered_df[columns_needed]

In [ ]:
filtered_df.head(10)

In [ ]:
# filter out any rows with missing values
filtered_df = filtered_df.dropna()
len(filtered_df)

In [ ]:
# cast the OP_CARRIER_AIRLINE_ID to int
filtered_df['OP_CARRIER_AIRLINE_ID'] = filtered_df['OP_CARRIER_AIRLINE_ID'].astype(int)

In [ ]:
filtered_df.head(10)

In [ ]:
# group by col and gather counts. Order by greatest to least
COL = 'DEP_TIME_BUCKET'
counts = filtered_df.groupby(COL).size().sort_values(ascending=False)
len(counts)

In [ ]:
counts

In [ ]:
SINGLE_CONDITIONAL_FILTER = 200
# group by orig airport id and gather counts. filter out any airports with less than 200 flights
orig_airport_counts = filtered_df.groupby('ORIGIN_AIRPORT_ID').size()
filtered_orig_airport_counts = orig_airport_counts[orig_airport_counts >= SINGLE_CONDITIONAL_FILTER]
filtered_df = filtered_df[filtered_df['ORIGIN_AIRPORT_ID'].isin(filtered_orig_airport_counts.index)]

In [ ]:
len(filtered_df)

In [ ]:
# do the same for destination airport
dest_airport_counts = filtered_df.groupby('DEST_AIRPORT_ID').size()
filtered_dest_airport_counts = dest_airport_counts[dest_airport_counts >= SINGLE_CONDITIONAL_FILTER]
filtered_df = filtered_df[filtered_df['DEST_AIRPORT_ID'].isin(filtered_dest_airport_counts.index)]

In [ ]:
len(filtered_df)

### Analysis
All above filtering logic done, we can now do analysis on this data frame!

In [61]:
cached_data_path = "data/cached_data.csv"

In [65]:
df = pd.read_csv(cached_data_path)

In [66]:
# re-index based on the first four columns for quick indexing later
CONDITIONAL_COLUMNS = [
    'OP_CARRIER_AIRLINE_ID',
    'ORIGIN_AIRPORT_ID',
    'DEST_AIRPORT_ID',
    'DEP_TIME_BUCKET',
]
df = df.set_index(CONDITIONAL_COLUMNS)

In [67]:
DELAY_COLUMN = 'ARR_DELAY'

In [68]:
# Group by the conditional columns and gather counts
grouped = df.groupby(CONDITIONAL_COLUMNS).size()
len(grouped)

60059

In [ ]:
# plot the distribution of counts in grouped
plt.figure(figsize=(12, 6))
plt.hist(grouped, bins=50, color='b', alpha=0.7)
plt.title('Distribution of Flight Counts by Group', fontsize=16)
plt.xlabel('Number of Flights', fontsize=12)
plt.ylabel('Frequency', fontsize=12)

In [ ]:
# figure out how many have size > 200
grouped[grouped > 200]

In [ ]:
# get the top 5 groups
top_groups = grouped[grouped > 200].sort_values(ascending=False).head(5)
top_groups

In [ ]:
df.groupby(['OP_CARRIER_AIRLINE_ID', 'ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID']).size().sort_values(ascending=False).head(5)

In [ ]:
df.groupby(['ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID']).size().sort_values(ascending=False).head(5)

In [ ]:
# filter for the to group from top_groups and plot the distribution of delays
top_group = top_groups.index[0]
top_group_df = df.loc[top_group]
plt.figure(figsize=(12, 6))
plt.hist(top_group_df[DELAY_COLUMN], bins=50, color='b', alpha=0.7, density=True)
plt.title(f"Distribution of Delays for Top Group: {top_group}", fontsize=16)
plt.xlabel('Delay (minutes)', fontsize=12)
plt.ylabel('Density', fontsize=12)

In [ ]:
top_group_df[top_group_df[DELAY_COLUMN] > 0].shape[0] / top_group_df.shape[0]

In [ ]:
filtered_df = top_group_df[top_group_df[DELAY_COLUMN] > 0]
plt.figure(figsize=(12, 6))
plt.hist(filtered_df[DELAY_COLUMN], bins=50, color='b', alpha=0.7, density=True)
plt.title(f"Distribution of Delays for Top Group: {top_group}", fontsize=16)
plt.xlabel('Delay (minutes)', fontsize=12)
plt.ylabel('Density', fontsize=12)

In [ ]:
# load lookup tables for airport
AIRPORT_LOOKUP_FILE = os.path.join('stored_data', 'lookup', 'L_AIRPORT_ID.csv')

# read the airport lookup table
airport_codes = pd.read_csv(AIRPORT_LOOKUP_FILE)
airport_codes.head(5)

In [ ]:
airport_codes = airport_codes.set_index('Code')

In [ ]:
travel_counts = df.groupby(['ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID']).size().sort_values(ascending=False)

# get top 50 travel counts
top_travel_counts = travel_counts.head(50)

# get the airport description for each origin and dest airport id in top_travel_counts
top_travel_counts = top_travel_counts.reset_index()
top_travel_counts['ORIGIN_AIRPORT'] = top_travel_counts['ORIGIN_AIRPORT_ID'].map(airport_codes['Description'])
top_travel_counts['DEST_AIRPORT'] = top_travel_counts['DEST_AIRPORT_ID'].map(airport_codes['Description'])

top_travel_counts = top_travel_counts.drop(columns=["ORIGIN_AIRPORT_ID", "DEST_AIRPORT_ID"])
top_travel_counts


## Ranking the Partition of the Four Conditional Variables in terms of Predicting full Conditional Distribution
We want an algorithm to model $P(D|C_1, C_2, C_3, C_4)$ the four conditional RVs when there's not enough data support.

To do this, we'll use a bayesian approach that assumes independence and conditional independence of certain subsets of $C_i$ with each other.

See derivation in whitepaper.

In this analysis, we'll take cases where the full conditional distribution has data support. Then, we rank the partitions on how well they model the full distribution. Finally, we average those ranks and rank the paritions based on their average rank score. 
During inference, we iterate through these partitions in order from highest rank to lowest until feasibility to model the query request.

In [116]:
asset_path = "app/backend/assets/model_params.csv"

In [117]:
df = pd.read_csv(asset_path)
df.head(10)

,OP_CARRIER_AIRLINE_ID,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,DEP_TIME_BUCKET,p,lambda
0,NaN,NaN,NaN,NaN,0.353679,0.024523
1,19393.0,NaN,NaN,NaN,0.400849,0.032757
2,19687.0,NaN,NaN,NaN,0.345942,0.041299
3,19690.0,NaN,NaN,NaN,0.427089,0.046093
4,19790.0,NaN,NaN,NaN,0.293436,0.025762
5,19805.0,NaN,NaN,NaN,0.379203,0.020860
6,19930.0,NaN,NaN,NaN,0.377557,0.035490
7,19977.0,NaN,NaN,NaN,0.345463,0.023433
8,20046.0,NaN,NaN,NaN,0.323010,0.017629
9,20225.0,NaN,NaN,NaN,0.540595,0.028692


In [118]:
# redefine col names
CONDITIONAL_COLUMNS = [
    'OP_CARRIER_AIRLINE_ID',
    'ORIGIN_AIRPORT_ID',
    'DEST_AIRPORT_ID',
    'DEP_TIME_BUCKET',
]

In [119]:
# convert to 'Int64' type for OP_CARRIER_AIRLINE_ID
df['OP_CARRIER_AIRLINE_ID'] = df['OP_CARRIER_AIRLINE_ID'].astype('Int64')

In [120]:
df = df.set_index(CONDITIONAL_COLUMNS, drop=False)

In [121]:
# filter for where the conditional columns are not NaN
fully_available_df = df.dropna(subset=CONDITIONAL_COLUMNS)

In [122]:
len(fully_available_df)

16740

In [93]:
def compute_bayesian_parameters(subset_params: np.ndarray, margin_params: np.ndarray) -> np.ndarray:
    num_subsets = len(subset_params)

    assert num_subsets > 0

    margin_lambda, margin_p = margin_params[0], margin_params[1]

    new_lambda = np.sum(subset_params[:, 0]) - margin_lambda * (num_subsets - 1)
    new_p = np.prod(subset_params[:, 0] * subset_params[:, 1]) / (margin_p * margin_lambda) ** (num_subsets - 1)
    new_p = new_p / new_lambda

    return np.array([new_lambda, new_p])



In [94]:
def kl_divergence(params_a: np.ndarray, params_b: np.ndarray) -> float:
    lambda_a, p_a = params_a[0], params_a[1]
    lambda_b, p_b = params_b[0], params_b[1]

    lambda_contribution = (lambda_b - lambda_a) * np.log(lambda_a / lambda_b)
    p_contribution = p_a * np.log(p_a / p_b)
    return lambda_contribution * p_contribution / lambda_a

In [95]:
# partition class of the four conditional columns
class Partition:
    def __init__(self, col_part: list) -> None:
        print(col_part)
        self.col_part = tuple(sorted(tuple(sorted(part) for part in col_part)))
    
    def __repr__(self) -> str:
        return repr(self.col_part)
    

In [100]:
from typing import List
def generate_partitions(arr: List[str]) -> List[List[str]]:
    if len(arr) == 1:
        return [[arr]]

    curr_elem = arr[0]
    next_partitions = generate_partitions(arr[1:])

    partitions = []
    for partition in next_partitions:
        for i, part in enumerate(partition):
            new_partition = partition.copy()
            new_partition[i] = part + [curr_elem]
            partitions.append(new_partition)
        partitions.append(partition + [[curr_elem]])
    
    return partitions

In [104]:
def generate_partitions_obj(arr: List[str]) -> List[Partition]:
    partitions = generate_partitions(arr)
    return [Partition(part) for part in partitions]

In [107]:
partitions = generate_partitions_obj(CONDITIONAL_COLUMNS)

[['DEP_TIME_BUCKET', 'DEST_AIRPORT_ID', 'ORIGIN_AIRPORT_ID', 'OP_CARRIER_AIRLINE_ID']]
[['DEP_TIME_BUCKET', 'DEST_AIRPORT_ID', 'ORIGIN_AIRPORT_ID'], ['OP_CARRIER_AIRLINE_ID']]
[['DEP_TIME_BUCKET', 'DEST_AIRPORT_ID', 'OP_CARRIER_AIRLINE_ID'], ['ORIGIN_AIRPORT_ID']]
[['DEP_TIME_BUCKET', 'DEST_AIRPORT_ID'], ['ORIGIN_AIRPORT_ID', 'OP_CARRIER_AIRLINE_ID']]
[['DEP_TIME_BUCKET', 'DEST_AIRPORT_ID'], ['ORIGIN_AIRPORT_ID'], ['OP_CARRIER_AIRLINE_ID']]
[['DEP_TIME_BUCKET', 'ORIGIN_AIRPORT_ID', 'OP_CARRIER_AIRLINE_ID'], ['DEST_AIRPORT_ID']]
[['DEP_TIME_BUCKET', 'ORIGIN_AIRPORT_ID'], ['DEST_AIRPORT_ID', 'OP_CARRIER_AIRLINE_ID']]
[['DEP_TIME_BUCKET', 'ORIGIN_AIRPORT_ID'], ['DEST_AIRPORT_ID'], ['OP_CARRIER_AIRLINE_ID']]
[['DEP_TIME_BUCKET', 'OP_CARRIER_AIRLINE_ID'], ['DEST_AIRPORT_ID', 'ORIGIN_AIRPORT_ID']]
[['DEP_TIME_BUCKET'], ['DEST_AIRPORT_ID', 'ORIGIN_AIRPORT_ID', 'OP_CARRIER_AIRLINE_ID']]
[['DEP_TIME_BUCKET'], ['DEST_AIRPORT_ID', 'ORIGIN_AIRPORT_ID'], ['OP_CARRIER_AIRLINE_ID']]
[['DEP_TIME_BUCKE

15

[(['DEP_TIME_BUCKET', 'DEST_AIRPORT_ID', 'OP_CARRIER_AIRLINE_ID', 'ORIGIN_AIRPORT_ID'],),
 (['DEP_TIME_BUCKET', 'DEST_AIRPORT_ID', 'ORIGIN_AIRPORT_ID'], ['OP_CARRIER_AIRLINE_ID']),
 (['DEP_TIME_BUCKET', 'DEST_AIRPORT_ID', 'OP_CARRIER_AIRLINE_ID'], ['ORIGIN_AIRPORT_ID']),
 (['DEP_TIME_BUCKET', 'DEST_AIRPORT_ID'], ['OP_CARRIER_AIRLINE_ID', 'ORIGIN_AIRPORT_ID']),
 (['DEP_TIME_BUCKET', 'DEST_AIRPORT_ID'], ['OP_CARRIER_AIRLINE_ID'], ['ORIGIN_AIRPORT_ID']),
 (['DEP_TIME_BUCKET', 'OP_CARRIER_AIRLINE_ID', 'ORIGIN_AIRPORT_ID'], ['DEST_AIRPORT_ID']),
 (['DEP_TIME_BUCKET', 'ORIGIN_AIRPORT_ID'], ['DEST_AIRPORT_ID', 'OP_CARRIER_AIRLINE_ID']),
 (['DEP_TIME_BUCKET', 'ORIGIN_AIRPORT_ID'], ['DEST_AIRPORT_ID'], ['OP_CARRIER_AIRLINE_ID']),
 (['DEP_TIME_BUCKET', 'OP_CARRIER_AIRLINE_ID'], ['DEST_AIRPORT_ID', 'ORIGIN_AIRPORT_ID']),
 (['DEP_TIME_BUCKET'], ['DEST_AIRPORT_ID', 'OP_CARRIER_AIRLINE_ID', 'ORIGIN_AIRPORT_ID']),
 (['DEP_TIME_BUCKET'], ['DEST_AIRPORT_ID', 'ORIGIN_AIRPORT_ID'], ['OP_CARRIER_AIRLINE_I

In [132]:
margin_df = df.loc[(None, None, None, None)]
margin_lambda = margin_df['lambda']
margin_p = margin_df['p']

margin_params = np.array([margin_lambda, margin_p])

In [126]:
partition_to_rank_lst = {
    part: [] for part in partitions
}

In [129]:
fully_available_df

OP_CARRIER_AIRLINE_ID  \
OP_CARRIER_AIRLINE_ID ORIGIN_AIRPORT_ID DEST_AIRPORT_ID DEP_TIME_BUCKET                          
19393                 10140.0           10423.0         afternoon                        19393   
                                                        morning                          19393   
                                        10800.0         afternoon                        19393   
                                        10821.0         afternoon                        19393   
                                        11259.0         afternoon                        19393   
...                                                                                        ...   
21167                 14893.0           12892.0         afternoon                        21167   
                                                        morning                          21167   
                                        14747.0         morning                          21167   
                      14908.0           14747.0         afternoon                        21167   
                                                        morning                          21167   

                                                                         ORIGIN_AIRPORT_ID  \
OP_CARRIER_AIRLINE_ID ORIGIN_AIRPORT_ID DEST_AIRPORT_ID DEP_TIME_BUCKET                      
19393                 10140.0           10423.0         afternoon                  10140.0   
                                                        morning                    10140.0   
                                        10800.0         afternoon                  10140.0   
                                        10821.0         afternoon                  10140.0   
                                        11259.0         afternoon                  10140.0   
...                                                                                    ...   
21167                 14893.0           12892.0         afternoon                  14893.0   
                                                        morning                    14893.0   
                                        14747.0         morning                    14893.0   
                      14908.0           14747.0         afternoon                  14908.0   
                                                        morning                    14908.0   

                                                                         DEST_AIRPORT_ID  \
OP_CARRIER_AIRLINE_ID ORIGIN_AIRPORT_ID DEST_AIRPORT_ID DEP_TIME_BUCKET                    
19393                 10140.0           10423.0         afternoon                10423.0   
                                                        morning                  10423.0   
                                        10800.0         afternoon                10800.0   
                                        10821.0         afternoon                10821.0   
                                        11259.0         afternoon                11259.0   
...                                                                                  ...   
21167                 14893.0           12892.0         afternoon                12892.0   
                                                        morning                  12892.0   
                                        14747.0         morning                  14747.0   
                      14908.0           14747.0         afternoon                14747.0   
                                                        morning                  14747.0   

                                                                        DEP_TIME_BUCKET  \
OP_CARRIER_AIRLINE_ID ORIGIN_AIRPORT_ID DEST_AIRPORT_ID DEP_TIME_BUCKET                   
19393                 10140.0           10423.0         afternoon             afternoon   
                                                        morning                 morning   
              

In [133]:
# iterate through all fully available data
for _, row in fully_available_df.iterrows():
    print(type(row))
    # get the p, lambda
    full_p, full_lambda = row['p'], row['lambda']

    # iterate through the partitions, derive the partition params, and compute its KL
    part_kl_scores = []  # tuple of part, kl_score
    for part in partitions:
        # gather the parameters for the partition
        part_params = []
        for subset in part.col_part:
            # filter df for the partition
            excluded_cols = list(set(CONDITIONAL_COLUMNS) - set(subset))
            part_row = df[df[subset] == row[subset] & df[excluded_cols].isna()]

            # extract out the p, lambda
            part_p, part_lambda = part_row['p'], part_row['lambda']
            part_params.append([part_lambda, part_p])

        # compute the new p, lambda
        new_params = compute_bayesian_parameters(np.array(part_params), margin_params)

        # compute the kl divergence
        kl_score = kl_divergence([full_lambda, full_p], new_params)

        part_kl_scores.append((part, kl_score))
    
    # sort by score
    part_kl_scores.sort(key=lambda x: x[1])

    # append to the partition_to_rank_lst
    for i, (part, kl_score) in enumerate(part_kl_scores):
        partition_to_rank_lst[part].append(i)


<class 'pandas.core.series.Series'>


ValueError: Can only compare identically-labeled (both index and columns) DataFrame objects